In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mlp import Layer, MultilayerPerceptron, SquaredError, Relu, Linear
from sklearn.metrics import r2_score

# -------------------- Loading MPG Dataset -------------------- #
class MPGDataLoader:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path

    def load_data(self):
        columns = ["mpg", "cylinders", "displacement", "horsepower", "weight",
                   "acceleration", "model year", "origin", "car name"]
        df = pd.read_csv(self.dataset_path, sep='\s+', names=columns, na_values="?")
        df.dropna(inplace=True)
        df.drop(columns=["car name"], inplace=True, errors='ignore')
        df["horsepower"] = pd.to_numeric(df["horsepower"], errors='coerce')
        df.dropna(inplace=True)

        y = df["mpg"].values.reshape(-1, 1)
        X = df.drop(columns=["mpg"]).values
        return X, y

# Load dataset
dataset_path = 'mpg-data/auto-mpg.data'
dataloader = MPGDataLoader(dataset_path)
X, y = dataloader.load_data()

# -------------------- Data Preprocessing -------------------- #
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train)
X_val = scaler_x.transform(X_val)
X_test = scaler_x.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# -------------------- Hyperparameter Tuning: Epoch -------------------- #
input_dim = X_train.shape[1]

epochs_list = [50, 100, 250, 500, 850]

results = {}

for epochs in epochs_list:
    print(f"\nTraining model with {epochs} epochs...")

    layers = [
        Layer(input_dim, 64, Relu(), dropout_rate=0.2),
        Layer(64, 64, Relu(), dropout_rate=0.2),
        Layer(64, 64, Relu(), dropout_rate=0.2),
        Layer(64, 1, Linear())
    ]

    mlp = MultilayerPerceptron(layers)
    train_losses, val_losses = mlp.train(
        X_train, y_train, X_val, y_val,
        SquaredError(), learning_rate=0.001, batch_size=32, epochs=epochs, optimizer='vanilla', momentum=0.7
    )

    y_pred = mlp.forward(X_test, training=False)
    y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))
    y_test_original = scaler_y.inverse_transform(y_test)

    r2 = r2_score(y_test_original, y_pred)

    results[epochs] = round(r2 * 100, 2)
    print(f"R² Score for {epochs} epochs: {results[epochs]}%")

# Identify Best Epoch Count
best_epochs = max(results, key=results.get)
best_r2 = results[best_epochs]

# -------------------- Summary of Results -------------------- #
print("\n===== Summary of Runs =====")
for ep, r2 in results.items():
    print(f"R² value for {ep} epochs: {r2}%")

print(f"\nBest Epochs: {best_epochs} epochs with R² = {best_r2}%")



Training model with 50 epochs...
Epoch 1/50 - Train Loss: 1.1005, Val Loss: 0.9078
Epoch 2/50 - Train Loss: 1.0447, Val Loss: 0.9105
Epoch 3/50 - Train Loss: 0.9586, Val Loss: 0.8242
Epoch 4/50 - Train Loss: 0.8482, Val Loss: 0.7458
Epoch 5/50 - Train Loss: 0.8049, Val Loss: 0.7007
Epoch 6/50 - Train Loss: 0.7621, Val Loss: 0.6407
Epoch 7/50 - Train Loss: 0.6885, Val Loss: 0.6284
Epoch 8/50 - Train Loss: 0.6867, Val Loss: 0.6066
Epoch 9/50 - Train Loss: 0.6054, Val Loss: 0.5958
Epoch 10/50 - Train Loss: 0.5872, Val Loss: 0.4785
Epoch 11/50 - Train Loss: 0.5109, Val Loss: 0.5054
Epoch 12/50 - Train Loss: 0.5099, Val Loss: 0.3956
Epoch 13/50 - Train Loss: 0.4378, Val Loss: 0.3963
Epoch 14/50 - Train Loss: 0.4426, Val Loss: 0.3525
Epoch 15/50 - Train Loss: 0.4406, Val Loss: 0.4758
Epoch 16/50 - Train Loss: 0.4050, Val Loss: 0.3481
Epoch 17/50 - Train Loss: 0.4051, Val Loss: 0.3252
Epoch 18/50 - Train Loss: 0.4239, Val Loss: 0.3626
Epoch 19/50 - Train Loss: 0.3688, Val Loss: 0.3335
Epoch 